<a href="https://colab.research.google.com/github/alkarps/GB.Methods-for-collecting-and-processing-data-from-the-Internet/blob/hw04/hw04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install mongodb > log

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
#!pip install pymongo

In [4]:
from pymongo import MongoClient
from pprint import pprint
from collections import namedtuple
import json

In [5]:
Vacancy = namedtuple("Vacancy", "source, name, link, min_salary, max_salary")
Salary = namedtuple("Salary", "amount, currency")

def create_salary(amount, currency):
  if isinstance(amount, str):
    return Salary(int(amount.replace("\u202f","")), currency.strip())
  else:
    return Salary(int(amount), currency.strip())

In [6]:
def to_entity(vacancy):
  def to_salary_entity(salary):
    if salary:
      return salary._asdict()
    return None
  if vacancy:
    _vacancy = vacancy._asdict()
    _vacancy['name_words'] = vacancy.name.split()
    _vacancy['min_salary'] = to_salary_entity(vacancy.min_salary)
    _vacancy['max_salary'] = to_salary_entity(vacancy.max_salary)
    return _vacancy
  return None

In [7]:
def from_entity(vacancy):
  def from_salary_entity(salary):
    if salary:
      return Salary(salary['amount'], salary['currency'])
    return None
  if vacancy:
    return Vacancy(vacancy['source'], vacancy['name'], vacancy['link'], from_salary_entity(vacancy['min_salary']), from_salary_entity(vacancy['max_salary']))
  return None

1. **Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.**
2. **Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)**
3. **Любая аналитика. Например, matching вакансий с разных площадок.**

In [8]:
def get_connect_hr_db():
  client = MongoClient('mongodb://127.0.0.1:27017/')
  return client.hr

In [9]:
def save_vacancies(vacancies):
  if vacancies is None or len(vacancies) == 0:
    return
  _vacancies = get_connect_hr_db().vacancies
  for vacancy in vacancies:
    if _vacancies.find_one({"link": vacancy.link}):
      continue
    else:
      entity = to_entity(vacancy)
      _vacancies.insert_one(entity)

In [10]:
def find_vacancies(name = None, min_salary = None, max_salary = None):
  def get_filter(name, min_salary, max_salary):
    filters = list()
    if name is not None:
      filters.append({ 'name_words': { '$in': name.split() } })
    if min_salary is not None:
      filters.append({ '$or': [{'min_salary.amount': {'$gte' : min_salary}}, {'min_salary': {'$exists': False} }, {'min_salary' : None }]})
    if max_salary is not None:
      filters.append({ '$or': [{'max_salary.amount': {'$lte' : max_salary}}, {'max_salary': {'$exists': False} }, {'max_salary' : None }]})
    if len(filters) == 0:
      return None
    elif len(filters) == 1:
      return filters[0]
    else:
      return {'$and': filters}
  _vacancies = get_connect_hr_db().vacancies
  filter = get_filter(name, min_salary, max_salary)
  if filter:
    vacancy_entities = _vacancies.find(filter)
  else: 
    vacancy_entities = _vacancies.find()
  return [from_entity(x) for x in vacancy_entities]

In [11]:
vacancies = [
    Vacancy('hh.ru', 'QA Engineer Auto (Java)', 'https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, None), 
    Vacancy('hh.ru', 'QA Engineer Auto (Java) double', 'https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, None), 
    Vacancy('hh.ru', 'QA Automation engineer (зарплатный проект)', 'https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, Salary(200000, 'руб.')), 
    Vacancy('hh.ru', 'Middle / Senior QA инженер', 'https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', Salary(200000, 'руб.'), None), 
    Vacancy('hh.ru', 'Senior QA инженер', 'https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', Salary(400000, 'руб.'), None), 
    Vacancy('hh.ru', 'Middle / Senior QA в команду hr-брендинговые продукты', 'https://hh.ru/vacancy/69811697?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80',  Salary(100000, 'руб.'), Salary(200000, 'руб.')), 
    Vacancy('hh.ru', 'Middle / Senior QA в команду безопасности', 'https://hh.ru/vacancy/47892571?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80',  None, Salary(500000, 'руб.')), 
    Vacancy('hh.ru', 'Middle / Senior QA инженер в команду mobi-dick', 'https://hh.ru/vacancy/71270946?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', None, None), 
    Vacancy('hh.ru', 'Ручной тестировщик мобильного приложения', 'https://hh.ru/vacancy/74386282?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', Salary(40000, 'руб.'), Salary(50000, 'руб.')), 
]

In [12]:
pprint(vacancies)

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Engineer Auto (Java) double', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер', link='https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=200000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search

In [13]:
get_connect_hr_db().vacancies.delete_many({})

In [14]:
save_vacancies(vacancies)

In [15]:
pprint(find_vacancies())

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер', link='https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=200000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=400000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Middle / Senior QA в команду hr-брендинговые продукты',

In [16]:
pprint(find_vacancies('QA'))

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер', link='https://hh.ru/vacancy/47759634?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=200000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=400000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Middle / Senior QA в команду hr-брендинговые продукты',

In [17]:
pprint(find_vacancies(min_salary=300000))

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=400000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Middle / Senior QA в команду безопасности', link='https://hh.ru/vacancy/47892571?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=500000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер в команду mob

In [18]:
pprint(find_vacancies('QA', 300000))

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=400000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Middle / Senior QA в команду безопасности', link='https://hh.ru/vacancy/47892571?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=500000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер в команду mob

In [19]:
pprint(find_vacancies('QA', 300000, 500000))

[Vacancy(source='hh.ru', name='QA Engineer Auto (Java)', link='https://hh.ru/vacancy/72076374?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=None),
 Vacancy(source='hh.ru', name='QA Automation engineer (зарплатный проект)', link='https://hh.ru/vacancy/73180600?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=200000, currency='руб.')),
 Vacancy(source='hh.ru', name='Senior QA инженер', link='https://hh.ru/vacancy/49887152?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=Salary(amount=400000, currency='руб.'), max_salary=None),
 Vacancy(source='hh.ru', name='Middle / Senior QA в команду безопасности', link='https://hh.ru/vacancy/47892571?from=vacancy_search_list&query=%D1%82%D0%B5%D1%81%D1%82%D0%B5%D1%80', min_salary=None, max_salary=Salary(amount=500000, currency='руб.')),
 Vacancy(source='hh.ru', name='Middle / Senior QA инженер в команду mob